In [1]:
import os
import re
import sls
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import geffnet
import logging
import fnmatch
import argparse
import torchvision
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
from sklearn import metrics
import torch.optim as optim
from datetime import datetime
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
from tensorboardX import SummaryWriter
import torch.utils.model_zoo as model_zoo
from efficientnet_pytorch import EfficientNet
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict, defaultdict
#from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, models, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import confusion_matrix,accuracy_score
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
data_dir = '/home/linh/Downloads/DeepWeeds/'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'

# Define your transforms for the training and testing sets
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomRotation(30),
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# Using the image datasets and the trainforms, define the data_loader
# batch_size = 64 for EfficientNet from B0 - B3
# batch_size = 32 for EfficientNet B4, B5
# batch_size = 16 for EfficientNet_B6
# batch_size = 8 for EfficientNet_B7
# batch_size = 32 for MixNet_s
batch_size = 16
data_loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4, pin_memory = True)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

print(dataset_sizes)
print(device)

"""# Label mapping
with open('/home/linh/Downloads/Derma/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)"""


'''f = open('/home/linh/Downloads/Derma/classes.txt','r')
cat_to_name = f.read()
print(cat_to_name)
f.close()
'''
### we get the class_to_index in the data_Set but what we really need is the cat_to_names  so we will create
_ = image_datasets['train'].class_to_idx
cat_to_name = {_[i]: i for i in list(_.keys())}

    
# Run this to test the data loader
images, labels = next(iter(data_loader['val']))
images.size()

{'train': 64125, 'val': 21375}
cuda:0


torch.Size([16, 3, 224, 224])

In [4]:
#model = EfficientNet.from_pretrained('efficientnet-b3')
model = create_model('tf_efficientnet_lite4', pretrained = True)
#from efficientnet_pytorch import EfficientNet
#model = EfficientNet.from_pretrained('efficientnet-b6')
# Create classifier
for param in model.parameters():
    param.requires_grad = True
n_classes = 9
model.classifier = nn.Linear(model.classifier.in_features, n_classes)

criterion = nn.CrossEntropyLoss()
#optimizer = Nadam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
optimizer = sls.Sls(model.parameters())
#optimizer = sls.Sls(model.parameters(), n_batches_per_epoch=len(data_loader['train']))
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [5]:
model.to(device)

def train_model(model, criterion, optimizer, 
                #scheduler, 
                num_epochs=200, checkpoint = None):
    since = time.time()

    if checkpoint is None:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = math.inf
        best_acc = 0.
    else:
        print(f'Val loss: {checkpoint["best_val_loss"]}, Val accuracy: {checkpoint["best_val_accuracy"]}')
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        #scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        best_loss = checkpoint['best_val_loss']
        best_acc = checkpoint['best_val_accuracy']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, (inputs, labels) in enumerate(data_loader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                # establish function for SLS optimization
                def closure():
                    probs = F.log_softmax(model(inputs), dim=1)
                    loss = F.nll_loss(probs, labels, reduction="sum")
                    
                    return loss
                
                if i % 1000 == 999:
                    print('[%d, %d] loss: %.8f' % 
                          (epoch + 1, i, running_loss / (i * inputs.size(0))))

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        #loss.backward()
                        optimizer.step(closure)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.8f} Acc: {:.8f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print(f'New best model found!')
                print(f'New record loss: {epoch_loss}, previous record loss: {best_loss}')
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'best_val_loss': best_loss,
                            'best_val_accuracy': best_acc,
                            #'scheduler_state_dict' : scheduler.state_dict(),
                            }, 
                            CHECK_POINT_PATH
                            )
                print(f'New record loss is SAVED: {epoch_loss}')
      
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.8f} Best val loss: {:.8f}'.format(best_acc, best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss, best_acc

In [ ]:
CHECK_POINT_PATH = '/home/linh/Downloads/DeepWeeds/weights/EfficientNet_Lite4_AP_SLS.pth'
try:
    checkpoint = torch.load(CHECK_POINT_PATH)
    print("checkpoint loaded")
except:
    checkpoint = None
    print("checkpoint not found")
if checkpoint == None:
    CHECK_POINT_PATH = CHECK_POINT_PATH
model, best_val_loss, best_val_acc = train_model(model,
                                                 criterion,
                                                 optimizer,
                                                 #scheduler,
                                                 num_epochs = 100,
                                                 checkpoint = None #torch.load(CHECK_POINT_PATH)
                                                 ) 
                                                
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss,
            'best_val_accuracy': best_val_acc,
            #'scheduler_state_dict': scheduler.state_dict(),
            }, CHECK_POINT_PATH)

checkpoint not found
Epoch 0/99
----------
[1, 999] loss: 0.85259798
[1, 1999] loss: 0.64890996
[1, 2999] loss: 0.54884999
[1, 3999] loss: 0.48149250
train Loss: 0.48111378 Acc: 0.83566472
[1, 999] loss: 0.18625599
val Loss: 0.18986047 Acc: 0.93861988
New best model found!
New record loss: 0.18986046899689568, previous record loss: inf
New record loss is SAVED: 0.18986046899689568

Epoch 1/99
----------
[2, 999] loss: 0.15698305
[2, 1999] loss: 0.15254926
[2, 2999] loss: 0.14496978
[2, 3999] loss: 0.13907779
train Loss: 0.13901545 Acc: 0.95345029
[2, 999] loss: 0.14480696
val Loss: 0.14847752 Acc: 0.95134503
New best model found!
New record loss: 0.14847752456236304, previous record loss: 0.18986046899689568
New record loss is SAVED: 0.14847752456236304

Epoch 2/99
----------
[3, 999] loss: 0.07657205
[3, 1999] loss: 0.07350872
[3, 2999] loss: 0.07020978
[3, 3999] loss: 0.06847991
train Loss: 0.06842704 Acc: 0.97808967
[3, 999] loss: 0.13714367
val Loss: 0.13702746 Acc: 0.95719298
New 